<a href="https://colab.research.google.com/github/Yara484/Fraud-Detection-Challenge/blob/main/Copy_of_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/Zindi project1   

/content/drive/My Drive/Zindi project1


In [ ]:
ls # List the files in the folder. Apparently, if you add the comment above or under the code it produces an error. Why?   

client_test.csv   invoice_test.csv   SampleSubmission.csv
client_train.csv  invoice_train.csv  submission.csv


In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import datetime
import gc
from IPython import display #What does this do?
%matplotlib inline 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

In [ ]:
client_train = pd.read_csv('/content/drive/My Drive/Zindi project1/client_train.csv')
invoice_train = pd.read_csv('/content/drive/My Drive/Zindi project1/invoice_train.csv')

client_test = pd.read_csv('/content/drive/My Drive/Zindi project1/client_test.csv')
invoice_test = pd.read_csv('/content/drive/My Drive/Zindi project1/invoice_test.csv')

submission_file = pd.read_csv('/content/drive/My Drive/Zindi project1/SampleSubmission.csv')

#df_train = pd.merge(client_train, invoice_train)
#df_test = pd.merge(client_test, invoice_test)
#df_train.head(10)

In [ ]:
client_train.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [ ]:
invoice_train.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC


In [ ]:
# Decide which categorical variables you want to use in model
for col_name in client_train.columns:
    if client_train[col_name].dtypes == 'object':
        unique_cat = len(client_train[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))
        

Feature 'client_id' has 135493 unique categories
Feature 'creation_date' has 8088 unique categories


In [ ]:
# Decide which categorical variables you want to use in model
for col_name in invoice_train.columns:
    if invoice_train[col_name].dtypes == 'object':
        unique_cat = len(invoice_train[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))

Feature 'client_id' has 135493 unique categories
Feature 'invoice_date' has 8275 unique categories
Feature 'counter_statue' has 16 unique categories
Feature 'counter_type' has 2 unique categories


In [ ]:
todummy_list = ['counter_statue', 'counter_type']

In [ ]:
# Function to dummy all the categorical variables used for modeling
def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False)
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [ ]:
invoice_train = dummy_df(invoice_train, todummy_list)
invoice_test = dummy_df(invoice_test, todummy_list)

In [ ]:
#changing the invoice date to datetime
for df in [invoice_train,invoice_test]:
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])

In [ ]:
#changing the creation date to datetime
for df in [client_train,client_test]:
    df['creation_date'] = pd.to_datetime(df['creation_date'])

In [ ]:
aggs = {}
aggs['consommation_level_1'] = ['mean']
aggs['consommation_level_2'] = ['mean']
aggs['consommation_level_3'] = ['mean']
aggs['consommation_level_4'] = ['mean']

In [ ]:
agg_trans = invoice_train.groupby(['client_id']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (invoice_train.groupby('client_id').size().reset_index(name='{}transactions_count'.format('1')))
agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [ ]:
train = pd.merge(client_train,agg_trans, on='client_id', how='left')

In [ ]:
agg_trans = invoice_test.groupby(['client_id']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (invoice_test.groupby('client_id').size().reset_index(name='{}transactions_count'.format('1')))
agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [ ]:
test = pd.merge(client_test,agg_trans, on='client_id', how='left')

In [ ]:
train.head()

,disrict,client_id,client_catg,region,creation_date,target,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,60,train_Client_0,11,101,1994-12-31,0.0,35,352.400000,10.571429,0.000000,0.000000
1,69,train_Client_1,11,107,2002-05-29,0.0,37,557.540541,0.000000,0.000000,0.000000
2,62,train_Client_10,11,301,1986-03-13,0.0,18,798.611111,37.888889,0.000000,0.000000
3,69,train_Client_100,11,105,1996-11-07,0.0,20,1.200000,0.000000,0.000000,0.000000
4,62,train_Client_1000,11,303,2014-10-14,0.0,14,663.714286,104.857143,117.357143,36.714286


In [ ]:
col_to_drop = ['client_id', 'creation_date']
for col in col_to_drop:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)

In [ ]:
train.head()

,disrict,client_catg,region,target,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,60,11,101,0.0,35,352.400000,10.571429,0.000000,0.000000
1,69,11,107,0.0,37,557.540541,0.000000,0.000000,0.000000
2,62,11,301,0.0,18,798.611111,37.888889,0.000000,0.000000
3,69,11,105,0.0,20,1.200000,0.000000,0.000000,0.000000
4,62,11,303,0.0,14,663.714286,104.857143,117.357143,36.714286


In [ ]:
test.head()

,disrict,client_catg,region,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,62,11,307,37,488.135135,3.243243,0.000000,0.000000
1,69,11,103,22,1091.409091,843.136364,182.318182,586.318182
2,62,11,310,74,554.040541,37.364865,15.743243,0.162162
3,60,11,101,40,244.350000,0.000000,0.000000,0.000000
4,62,11,301,53,568.188679,145.056604,33.679245,0.000000


In [ ]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135493 entries, 0 to 135492
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   disrict                    135493 non-null  int64  
 1   client_catg                135493 non-null  int64  
 2   region                     135493 non-null  int64  
 3   target                     135493 non-null  float64
 4   1transactions_count        135493 non-null  int64  
 5   consommation_level_1_mean  135493 non-null  float64
 6   consommation_level_2_mean  135493 non-null  float64
 7   consommation_level_3_mean  135493 non-null  float64
 8   consommation_level_4_mean  135493 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 10.3 MB
None


In [ ]:
train.isnull().sum().sort_values(ascending=False).head()

consommation_level_4_mean    0
consommation_level_3_mean    0
consommation_level_2_mean    0
consommation_level_1_mean    0
1transactions_count          0
dtype: int64

In [ ]:
test.isnull().sum().sort_values(ascending=False).head()

consommation_level_4_mean    0
consommation_level_3_mean    0
consommation_level_2_mean    0
consommation_level_1_mean    0
1transactions_count          0
dtype: int64

In [ ]:
target=train['target']
train.drop('target',axis=1,inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train, target)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
rf_random.best_params_

In [ ]:
rf_random = rf_random.predict_proba(test)

In [ ]:
rf_random = pd.DataFrame(rf_random)

In [ ]:
rf_random.head()

In [ ]:
# create submission DataFrame
submission2 = pd.DataFrame({"client_id": submission_file["client_id"], "target": rf_random[1]})

In [ ]:
# Create submission csv file csv file
submission2.to_csv('/content/drive/My Drive/Zindi project1/submission_file.csv', index = False)

In [ ]:
submission2.head()